# Heart disease classification algoritm

### Read data

In [2]:
import pandas as pd
import numpy as np

In [4]:
heart=pd.read_csv("heart.csv")
heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [7]:
heart.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [12]:
heart['target'].value_counts()

1    526
0    499
Name: target, dtype: int64

In [18]:
heart.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [29]:
heart['fbs'].value_counts()

0    872
1    153
Name: fbs, dtype: int64

### Train-Test split

In [36]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(heart,heart['fbs']):
    train_set=heart.iloc[train_index]
    test_set=heart.iloc[test_index]

In [37]:
train_set['fbs'].value_counts()

0    698
1    122
Name: fbs, dtype: int64

In [38]:
test_set['fbs'].value_counts()

0    174
1     31
Name: fbs, dtype: int64

### Creating a pipeline

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
my_pipeline=Pipeline(
[
    ("imputer",SimpleImputer(strategy="median"))
])

### Features and label

In [50]:
train_features = train_set.drop("target",axis=1)
train_target = train_set['target'].copy()

In [51]:
train_features.shape

(820, 13)

In [52]:
train_target.shape

(820,)

In [53]:
train_features=my_pipeline.fit_transform(train_features)

### Selecting a desired model

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#model=LogisticRegression()
#model=KNeighborsClassifier()
model=DecisionTreeClassifier()
#model=RandomForestClassifier()

In [73]:
model.fit(train_features,train_target)

DecisionTreeClassifier()

In [74]:
pred=model.predict(train_features)

In [75]:
from sklearn.metrics import accuracy_score,confusion_matrix

accuracy_score(train_target,pred)

1.0

In [76]:
confusion_matrix(train_target,pred)

array([[396,   0],
       [  0, 424]], dtype=int64)

### Dumping model

In [77]:
from joblib import dump
dump(model,"heart_disease.joblib")

['heart_disease.joblib']

In [78]:
%store test_set

Stored 'test_set' (DataFrame)


In [79]:
%store my_pipeline

Stored 'my_pipeline' (Pipeline)
